<a href="https://colab.research.google.com/github/worldwidekatie/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module1-rnn-and-lstm/LS_DS_431_RNN_and_LSTM_Assignment_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
import requests
import pandas as pd

In [2]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [3]:
#Shakespeare Data Parsed by Play
df_toc.head()

,title,start,end,text
0,ALL’S WELL THAT ENDS WELL,2777,7738,ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nConte...
1,THE TRAGEDY OF ANTONY AND CLEOPATRA,7739,11840,THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\nDRA...
2,AS YOU LIKE IT,11841,14631,AS YOU LIKE IT\r\n\r\nDRAMATIS PERSONAE.\r\n\r...
3,THE COMEDY OF ERRORS,14632,17832,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
4,THE TRAGEDY OF CORIOLANUS,17833,27806,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...


In [4]:
 genre = ["COMEDY",
 "TRAGEDY",
 "COMEDY",
 "COMEDY",
 "TRAGEDY",
 "COMEDY",
 "TRAGEDY",
 "HISTORY",
 "HISTORY",
 "HISTORY",
 "HISTORY",
 "HISTORY",
 "HISTORY",
 "HISTORY",
 "HISTORY",
 "TRAGEDY",
 "TRAGEDY",
 "COMEDY",
 "TRAGEDY",
 "COMEDY",
 "COMEDY",
 "COMEDY",
 "COMEDY",
 "COMEDY",
 "TRAGEDY",
 "COMEDY",
 "HISTORY",
 "HISTORY",
 "TRAGEDY",
 "COMEDY",
 "COMEDY",
 "TRAGEDY",
 "TRAGEDY",
 "COMEDY",
 "COMEDY",
 "COMEDY",
 "TRAGEDY",
 "COMEDY",
 "POETRY",
 "POETRY",
 "POETRY",
 "POETRY",
 "POETRY"]

In [5]:
df_toc['genre'] = genre
df_toc['target'] = df_toc['genre'].replace({"COMEDY": 1, "TRAGEDY": 2, "HISTORY": 3, "POETRY": 4})
df_toc.head(2)

,title,start,end,text,genre,target
0,ALL’S WELL THAT ENDS WELL,2777,7738,ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nConte...,COMEDY,1
1,THE TRAGEDY OF ANTONY AND CLEOPATRA,7739,11840,THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\nDRA...,TRAGEDY,2


# Train an RNN
Use the above as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

In [6]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import pandas as pd
import random
import sys

In [18]:
# Encode Data as Chars
text = " ".join(df_toc['text'].values)

# Unique Characters
chars = list(set(text))

# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [21]:
len(chars)

106

In [22]:
# Create the sequence data

maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  1127161


In [23]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [25]:
x.shape

(1127161, 40, 106)

In [26]:
y.shape

(1127161, 106)

In [7]:
X_train = df_toc['text']
y_train = df_toc['target']

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
# INSTANTIATE
vect = TfidfVectorizer()

# FIT
vect.fit(X_train)

# TRANSFORM
dtm = vect.transform(X_train)

dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())


In [16]:
x_train = vect.transform(X_train)

In [38]:
dtm.head()

,000,10,100,1000,1004,1009,1012,1016,1020,1024,1028,1033,1036,104,1040,1044,1049,1053,1057,1060,1065,1069,1072,1078,108,1081,1085,1088,1093,1096,1100,1105,1108,1112,1116,112,1120,1124,1129,1132,...,yourself,yourselves,youth,youthes,youthful,youths,youtli,yron,yrons,yslaked,ythee,zanies,zany,zeal,zeale,zealous,zeals,zed,zenelophon,zenith,zephyrs,zip,zir,zo,zodiac,zodiacs,zone,zounds,zwaggered,zwounds,æacides,æneas,æson,échapper,écolier,écoutez,écus,été,êtes,être
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004813,0.001293,0.006270,0.0,0.000819,0.0,0.0,0.0,0.0,0.0,0.007069,0.0,0.0,0.0,0.0,0.001414,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.006857,0.000553,0.002234,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004600,0.000618,0.014484,0.0,0.001566,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003251,0.000000,0.000529,0.0,0.000830,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004503,0.002723,0.004156,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
dtm.shape

(43, 26329)

In [27]:
model = Sequential()

model.add(Embedding(1127161, 106))
model.add(LSTM(106, dropout=0.2, recurrent_dropout=0.2)) #return_squences=True, this does something when replacing dropout and recurrent dropout
#model.add(Dense(1, activation='sigmoid'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 106)         119479066 
_________________________________________________________________
lstm_1 (LSTM)                (None, 106)               90312     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 428       
Total params: 119,569,806
Trainable params: 119,569,806
Non-trainable params: 0
_________________________________________________________________


In [28]:
unicorns = model.fit(x, y,
          batch_size=500, 
          epochs=1)

ValueError: ignored

In [44]:

model = Sequential()
model.add(Embedding(26329, 1))
model.add(LSTM(100)
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(dtm, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

SyntaxError: ignored

In [29]:
from tensorflow.features.text import SubwordTextEncoder

ModuleNotFoundError: ignored

In [14]:
# build the model: a single LSTM

model = Sequential()
model.add(LSTM(106, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

## Generate Text

Then, use that trained RNN to generate Shakespearean-ish text. **Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.**

In [15]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [17]:
# fit the model

model.fit(x, y,
          batch_size=200,
          epochs=10, #changed to five for time's sake
          callbacks=[print_callback])

Epoch 1/10
5632/5636 [============================>.] - ETA: 0s - loss: 2.2317
----- Generating text after Epoch: 0
----- Generating with seed: " he hath said to you?

OPHELIA.
So pl"
 he hath said to you?

OPHELIA.









To than stow in tale son the pleve in tree
5636/5636 [==============================] - 41s 7ms/step - loss: 2.2315
Epoch 2/10
5632/5636 [============================>.] - ETA: 0s - loss: 1.8909
----- Generating text after Epoch: 1
----- Generating with seed: "CLES.
O you gods!
Why do you make us l"
CLES.
O you gods!









  heaven an
5636/5636 [==============================] - 41s 7ms/step - loss: 1.8908
Epoch 3/10
5633/5636 [============================>.] - ETA: 0s - loss: 1.7836
----- Generating text after Epoch: 2
----- Generating with seed: "    If with the sap of reason you would "









    The ushombus good hath, ic shas’
5636/5636 [==============================] - 42s 7ms/step - loss: 1.7835
Epoch 4/10
5635/5636 [============================>.] - 

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN